# Use Azure API with Cohere Command R 08-2024, Cohere Command R+ 08-2024, Command R and Command R+

This notebook shows examples of how to use Cohere Command R 08-2024, Cohere Command R+ 08-2024, Command R and Command R+ APIs offered by Microsoft Azure AI and Azure ML. 

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere) for the Cohere family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisite

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Cohere Command R 08-2024`, `Cohere Command R+ 08-2024`,`Cohere Command R`, `Cohere Command R+`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere) for model deployment and inference.

## HTTP Requests API Usage

Models deployed in Azure AI and Azure ML exposes 2 different APIs you can consume:

* Cohere API
* Azure AI inference API

- The **Cohere API** is designed to work with Cohere tooling specific tooling and it follows the Cohere public documentation.
- The **Azure AI inference API** is designed to work with Azure AI compatible tooling. This API is also OpenAI compatible.

The following examples shows how to use Azure AI inference API. For details about Cohere API you can see [Chat (Cohere docs)](https://docs.cohere.com/reference/chat).

### Basics

For using the REST API, You will need to have a Endpoint url and Authentication Key associated with that endpoint.  
This can be acquired from previous steps.  

In this chat completion example, we use a simple curl call for illustration. There are three major components:  

* The `host-url` is your endpoint url with chat completion schema `/v1/chat/completions`. 
* The `headers` defines the content type as well as your api key. 
* The `payload` or `data`, which is your prompt detail and model hyper parameters.

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 50}'

### Streaming

One fantastic feature the API offered is the streaming capability. Streaming allows the generated tokens to be sent as data-only server-sent events whenever they become available. This is extremely important for interactive applications such as chatbots, so the user is always engaged.  

To use streaming, simply set `"stream":"True"` as part of the request payload.  
In the streaming mode, the REST API response will be different from non-streaming mode.

Here is an example: 

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 500, "stream": "True"}'

As you can see the result comes back as a stream of `data` objects, each contains generated information including a `choice`.  
The stream terminated by a `data:[DONE]\n\n` message.

### Content Safety Filtering

All Azure Cohere endpoints will have content safety feature turned on. Both input prompt and output tokens are filtered by this service automatically.  
To know more about the impact to the request/response payload, please refer to official guide [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/content-filter?tabs=python).   

For model input and output, if the filter detected there is harmful content. The generation will error out with reponse payload containing the reasoning, along with which type of content violation it is and severity.  

Here is an example prompt that triggered content safety filtering:


In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"How to make bomb?","role":"user"}], "max_tokens": 50}'

## HTTP Requests API Usage in Python

Besides calling the API directly from command line tools. You can also programatically call them in Python. Here there is a chat completion example:




In [ ]:
import urllib.request
import json

# Configure payload data sending to API endpoint
data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}

body = str.encode(json.dumps(data))

# Replace the url with your API endpoint
url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"

# Replace this with the key for the endpoint
api_key = "your-auth-key"
if not api_key:
    raise Exception("API Key is missing")

headers = {"Content-Type": "application/json", "Authorization": (api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

However in this example, the streamed data content returns back as a single payload. It didn't stream as a serial of data events as we wished. To build true streaming capabilities utilizing the API endpoint, we will utilize [`requests`](https://requests.readthedocs.io/en/latest/) library instead.

### Streaming in Python

`Requests` library is a simple HTTP library for Python built with [`urllib3`](https://github.com/urllib3/urllib3). It automatically maintains the keep-alive and HTTP connection pooling. With the `Session` class, we can easily stream the result from our API calls.  

Here is a quick example:

In [ ]:
import json
import requests

data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}


def post_stream(url):
    s = requests.Session()
    api_key = "your-auth-key"
    headers = {"Content-Type": "application/json", "Authorization": (api_key)}

    with s.post(url, data=json.dumps(data), headers=headers, stream=True) as resp:
        print(resp.status_code)
        for line in resp.iter_lines():
            if line:
                print(line)


url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"
post_stream(url)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
